In [1]:
import os
import numpy as np
import ast
from order_fulfillment_environment import OrderFulfillment
from LP_fulfillment_item_facility_based import ItemFacilityLPSolver
from fulfillment_policy_Will_Ma import WillMaFulfillmentPolicy
from fulfillment_policy import FulfillmentPolicy

In [2]:
num_items = 20
n_max = 5
n_0 = 5
p_stock = 0.75
CSL=0.5
T = 10**2

facility_path = "fulfillment_centers_warmup.csv"
cities_path = "cities_warmup.csv"
home_path = "/Users/ayoubamil/Documents/Documents/GitHub/fulfillment-magician_AA_AM_YW/ORDER_FULFILLMENT/Data"
facilities_path = os.path.join(home_path, facility_path)
cities_path =  os.path.join(home_path, cities_path)

In [3]:
def DLP_WillMa(order_fulfillment, print_optimal_values=False):
    
    solver = ItemFacilityLPSolver(order_fulfillment)

    results = solver.optimize()

    # Extract the results
    optimal_u = results["optimal_u"]
    optimal_y = results["optimal_y"]
    num_vars = results["num_vars"]
    num_constrs = results["num_constrs"]
    optimal_value = results["optimal_value"]

    # Step 4: Check if optimization was successful and print results
    if optimal_u and optimal_y:
        
        if print_optimal_values:
            print("Optimal 'u' values:")
            for var_name, value in optimal_u.items():
                print(f"{var_name} = {value}")

            print("\nOptimal 'y' values:")
            for var_name, value in optimal_y.items():
                print(f"{var_name} = {value}")

        return num_vars, num_constrs, optimal_value, optimal_u, optimal_y
    
    else:
        print("Optimization did not find an optimal solution for Will Ma's formulation.")

In [4]:
def DLP_ours(fulfillment_policy, print_optimal_values=False):
    # Call the method and get the results
    results = fulfillment_policy.get_optimization_results

    # Check if the optimization was successful and optionally print the results
    if results is not None:
        if print_optimal_values:
            print("Optimal 'x' values:")
            for q, x_values in results["x_values"].items():
                print(f"Order Type {q}:")
                for method_index, value in x_values.items():
                    print(f"  Method {method_index}: {value}")

            print(f"\nNumber of decision variables: {results['num_vars']}")
            print(f"Number of constraints: {results['num_constrs']}")
            print(f"Optimal objective value: {results['optimal_value']}")

        return results["num_vars"], results["num_constrs"], results["optimal_value"]
    
    else:
        print("Optimization did not find an optimal solution for our formulation.")
        return None

In [5]:
def calculate_average_costs(num_instances, num_order_sequences, n_max_values, conservative_prob):
    
    wm_DLP = {}
    our_DLP = {}
    simulated_average_costs = {}

    for n_max in n_max_values:
        
        wm_number_of_variables = []
        wm_number_of_constraints = []
        wm_optimal_values = []
        
        our_number_of_variables = []
        our_number_of_constraints = []
        our_optimal_values = []
        
        wm_average_costs = []
        our_average_costs = []
        
        for instance in range(1, num_instances + 1):

            order_fulfillment = OrderFulfillment(num_items=num_items, n_max=n_max, n_0=n_0, p_stock=p_stock, T=T, CSL=CSL, 
                                    facilities_data=facilities_path, 
                                    cities_data=cities_path, 
                                    prob_seed_value=instance, 
                                    order_seed_value=instance, 
                                    inv_seed_value=instance)
            fulfillment_policy = FulfillmentPolicy(num_items=num_items, n_max=n_max, n_0=n_0, p_stock=p_stock, T=T, CSL=CSL, 
                                    facilities_data=facilities_path, 
                                    cities_data=cities_path, 
                                    prob_seed_value=instance, 
                                    order_seed_value=instance, 
                                    inv_seed_value=instance)
            
            wm_num_vars, wm_num_constrs, wm_optimal_value, optimal_u, optimal_y = DLP_WillMa(order_fulfillment)
            wm_number_of_variables.append(wm_num_vars)
            wm_number_of_constraints.append(wm_num_constrs)
            wm_optimal_values.append(wm_optimal_value)
            
            our_num_vars, our_num_constrs, our_optimal_value = DLP_ours(fulfillment_policy)
            our_number_of_variables.append(our_num_vars)
            our_number_of_constraints.append(our_num_constrs)
            our_optimal_values.append(our_optimal_value)
            
            # Initialize costs for each combination of instance and n_max
            total_our_cost = 0
            total_will_ma_cost = 0

            for seed_order_sequence in range(1, num_order_sequences + 1):
                # Run the fulfillment policies with the updated seed
                inventory_consumption = fulfillment_policy.initialize_inventory_consumption()
                magician_problems = fulfillment_policy.generate_magician_problems(conservative_prob=conservative_prob)

                sampled_orders_index, sampled_orders, sampled_methods, accepts_decisions, fulfillment_costs = fulfillment_policy.new_fulfillment_policy(
                    inventory_consumption, magician_problems, seed_value=seed_order_sequence)
                fulfillment_policy.check_consistency(inventory_consumption)

                our_cost = sum(fulfillment_costs)
                total_our_cost += our_cost

                will_ma_policy = WillMaFulfillmentPolicy(order_fulfillment, optimal_u)
                will_ma_cost = will_ma_policy.run(sampled_orders, seed_value=seed_order_sequence)
                total_will_ma_cost += will_ma_cost

            # Calculate average costs for the current combination of n_max and instance
            average_our_cost = total_our_cost / num_order_sequences
            average_will_ma_cost = total_will_ma_cost / num_order_sequences

            our_average_costs.append(average_our_cost)
            wm_average_costs.append(average_will_ma_cost)

        wm_DLP[n_max] = {"number_of_variables": wm_number_of_variables, "number_of_constraints": wm_number_of_constraints, "optimal_value": wm_optimal_value}
        our_DLP[n_max] = {"number_of_variables": our_number_of_variables, "number_of_constraints": our_number_of_constraints, "optimal_value": our_optimal_value}
        simulated_average_costs[n_max] = {"our_average_costs": our_average_costs, "wm_average_costs": wm_average_costs}

    return wm_DLP, our_DLP, simulated_average_costs    

In [6]:
simulation = calculate_average_costs(num_instances = 30, num_order_sequences = 30, n_max_values = [2, 5], conservative_prob = 0.01)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-12
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken w

In [7]:
with open('new_algo_compare_costs_results_T=100_x=0.01.txt', 'w') as file:
    file.write(str(simulation))

In [9]:
# saved_simulation = 'new_algo_compare_costs_results_T=100_x=0.01.txt'

# # Step 1: Read the Text File
# with open(saved_simulation, 'r') as file:
#     data_str = file.read()

# # Step 2: Parse the Data
# data = ast.literal_eval(data_str)

# # Step 3: Organize the Data
# wm_DLP = {}
# our_DLP = {}
# simulated_average_costs = {}

# for n_max in data.keys():
#     wm_DLP[n_max] = {"number_of_variables": data[n_max]['number_of_variables'],
#                      "number_of_constraints": data[n_max]['number_of_constraints'],
#                      "optimal_value": data[n_max]['optimal_value']}
    
#     our_DLP[n_max] = {"number_of_variables": data[n_max]['number_of_variables'],
#                       "number_of_constraints": data[n_max]['number_of_constraints'],
#                       "optimal_value": data[n_max]['optimal_value']}
    
#     simulated_average_costs[n_max] = {"our_average_costs": data[n_max]['our_average_costs'],
#                                       "wm_average_costs": data[n_max]['wm_average_costs']}

In [26]:
n_max = 5
simulation[0][n_max]["optimal_value"]

936.1783686961312

In [27]:
simulation[1][n_max]["optimal_value"]

959.546003253489

In [28]:
np.mean(simulation[2][n_max]["our_average_costs"])

1421.4748894624618

In [29]:
np.mean(simulation[2][n_max]["wm_average_costs"])

1488.730172169522

In [ ]:
# value = 0
# for instance in range(1, 31):    
#     order_fulfillment = OrderFulfillment(num_items=num_items, n_max=n_max, n_0=n_0, p_stock=p_stock, T=T, CSL=CSL, 
#                                         facilities_data=facilities_path, 
#                                         cities_data=cities_path, 
#                                         prob_seed_value=instance, 
#                                         order_seed_value=instance, 
#                                         inv_seed_value=instance, 
#                                         alpha=alpha)

#     value += min(y for x in order_fulfillment.safety_stock for y in x if y>0)
# value = value/30
# value